# Functional API

*This is a companion notebook for the excellent book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff) ([code provided by François Chollet](https://github.com/fchollet/deep-learning-with-python-notebooks)).* 

The Sequential model is easy to use, but its applicability is extremely limited: it can
only express models with a single input and a single output, applying one layer after
the other in a sequential fashion. 

In practice, it’s pretty common to encounter models
with multiple inputs (say, an image and its metadata), multiple outputs (different
things you want to predict about the data), or a nonlinear topology. In such cases, you’d build your model using the Functional API. 

## Setup

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np

## Simple example

A simple Functional model with two `Dense` layers:

In [ ]:
# Declare an input (sample has shape 3,1)
inputs = keras.Input(shape=(___,), name="my_input")

# Create a layer with 64 nodes and call it on the input
features = layers.Dense(___, activation="relu")(___)

# Create final output layer with 10 classes and call it on the features
outputs = layers.Dense(___, activation="___")(___)

# Specify inputs and outputs in the Model constructor
model = keras.Model(inputs=___, outputs=___)

Let's take a look at the objects:

In [ ]:
inputs.shape

- The number of samples per batch is variable (indicated by the `None` batch size).
- The model will process batches where each sample has shape `(3,)` (wich is the same as `(3,1)`).

In [ ]:
inputs.dtype

- The batches will have dtype `float32`. 

We call such an object a symbolic tensor. It doesn’t contain any actual data, but it
encodes the specifications of the actual tensors of data that the model will see when
you use it. It stands for future tensors of data.

In [ ]:
features.shape

Summary of the model:

In [ ]:
model.summary()

## Multi-input, multi-output

Unlike the previous toy model, most deep learning models don’t look like lists—they look like
graphs. 

They may, for instance, have multiple inputs or multiple outputs. 

It’s for this kind of model that the Functional API really shines. 

Let’s say you’re building a system to rank customer support tickets by priority and route them to the appropriate department. 

**Your model has three inputs:**

1. The title of the ticket (text input)
1. The text body of the ticket (text input)
1. Any tags added by the user (categorical input, assumed here to be one-hot encoded)

We can encode the text inputs as arrays of ones and zeros of size `vocabulary_size`

**Your model also has two outputs:**

- The priority score of the ticket, a scalar between 0 and 1 (sigmoid output)
- The department that should handle the ticket (a softmax over the set of departments)

You can build this model in a few lines with the Functional API.

In [ ]:
# 1) DEFINE VARIABLES

# Size of vocabulary obtained when preprocessing text data: 10000
vocabulary_size = ___

# Number of unique issue tags: 100
num_tags = ___

# Number of departments for predictions: 4
num_departments = ___

In [ ]:
# 2) DEFINE MODEL INPUTS

# Title of tickets (shape of vocabulary_zize)
title = keras.Input(shape=(___,), name="title")

# Body of the tickets
text_body = keras.Input(shape=(___,), name="text_body")

# Tags added by user
tags = keras.___(shape=(___,), name="tags")

In [ ]:
# 3) FEATURES

# Combine inputs 
features = layers.Concatenate()([___, ___, ___])

# Intermediate layer with 64 nodes and relu activation (call on features)
features = layers.Dense(___, activation="___")(___)

In [ ]:
# 4) DEFINE MODEL OUTPUTS

# Priority score of the ticket (score between 0 and 1); use sigmoid; 
# call it priority; call on features
priority = layers.Dense(___, activation="___", name="___")(___)

# Department that should handle the ticket
department = layers.Dense(___, activation="___", name="department")(___)

In [ ]:
# 5) CREATE MODEL

# Specify inputs and outputs
model = keras.Model(inputs=[___, ___, ___], outputs=[___, ___])

### Training

Option 1: Providing lists of input & target arrays

- First, we create some random input data as well as random data for our labels. 
- We will use this data to train our model:

In [ ]:
# Define total number of samples (100)
num_samples = ___

# Create random input data
title_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
text_body_data = np.random.randint(0, 2, size=(num_samples, vocabulary_size))
tags_data = np.random.randint(0, 2, size=(num_samples, num_tags))

# Create random labels
priority_data = np.random.random(size=(num_samples, 1))
department_data = np.random.randint(0, 2, size=(num_samples, num_departments))

In [ ]:
# Compile model (use rmsprop optimizer; 
# loss = mean_squared_error and categorical_crossentropy
# metrics = mean_absolute_error and accuracy
model.compile(optimizer="___",
              loss=["___", "___"],
              metrics=[["___"], ["___"]])

# Fit model to data (define input and output)
# use only 1 epoch
model.fit([___],
          [___],
          epochs=___)

# Evaluate model
model.evaluate([___],
               [___])

# Make predictions
priority_preds, department_preds = model.predict([___])

Option 2: Providing dicts of input & target arrays

If you don’t want to rely on input order (for instance, because you have many inputs
or outputs), you can also leverage the names you gave to the Input objects and the
output layers, and pass data via dictionaries.

In [ ]:
model.compile(optimizer="rmsprop",
              loss={"priority": "mean_squared_error", "department": "categorical_crossentropy"},
              metrics={"priority": ["mean_absolute_error"], "department": ["accuracy"]})

model.fit({"title": title_data, "text_body": text_body_data, "tags": tags_data},
          {"priority": priority_data, "department": department_data},
          epochs=1)

model.evaluate({"title": title_data, "text_body": text_body_data, "tags": tags_data},
               {"priority": priority_data, "department": department_data})

priority_preds, department_preds = model.predict(
    {"title": title_data, "text_body": text_body_data, "tags": tags_data})

### Access layers

A Functional model is an explicit graph data structure. This makes it possible to
inspect how layers are connected and reuse previous graph nodes (which are layer
outputs) as part of new models. This enables two important use cases: 

- model visualization and 
- feature extraction.

Let’s visualize the connectivity of the model we just defined (the topology of the
model). You can plot a Functional model as a graph with the plot_model():

In [ ]:
keras.utils.plot_model(model, "ticket_classifier.png")

You can add to this plot the input and output shapes of each layer in the model, which
can be helpful during debugging:

In [ ]:
keras.utils.plot_model(model, "ticket_classifier_with_shape_info.png", show_shapes=True)

The “None” in the tensor shapes represents the batch size: this model allows batches
of any size.

### Retrieving inputs or outputs

- Access to layer connectivity also means that you can inspect and reuse individual nodes (layer calls) in the graph. 
- The model.layers model property provides the list of layers that make up the model, and for each layer you can query layer.input and layer.output

In [ ]:
model.layers

In [ ]:
model.layers[3].input

In [ ]:
model.layers[3].output

This enables you to do 

- feature extraction: creating models that reuse intermediate features from another model.

- Estimate how long a given issue ticket will take to resolve, a kind of difficulty rating. 

- You could do this via a classification layer over three categories: “quick,” “medium,” and “difficult.” 

- You don’t need to recreate and retrain a model from scratch. 
- You can start from the intermediate features of your previous model, since you have access to them, like this.

### Creating a new model

Creating a new model by reusing intermediate layer outputs:

In [ ]:
# layers[4] is our intermediate Dense layer
features = model.layers[4].output

difficulty = layers.Dense(3, activation="softmax", name="difficulty")(features)

new_model = keras.Model(
    inputs=[title, text_body, tags],
    outputs=[priority, department, difficulty])

In [ ]:
keras.utils.plot_model(new_model, "updated_ticket_classifier.png", show_shapes=True)